In [1]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
import logging
from langchain.chains import LLMChain
import os
import time
from dotenv import load_dotenv

load_dotenv()

# Configuración del logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def tiempo_de_ejecucion(func):
    """
    Decorador que registra el tiempo de ejecución de una función.
    """
    def wrapper(*args, **kwargs):
        inicio = time.time()
        resultado = func(*args, **kwargs)
        fin = time.time()
        logging.info(f"{func.__name__} tomó {fin - inicio} segundos en ejecutarse.")
        return resultado
    return wrapper

In [2]:
# Example usage

OpenAI.api_key = os.getenv("OPENAI_API_KEY")
json_path = '/Users/adrianinfantes/Desktop/AIR/CollegeStudies/MachineLearningPath/Portfolio/DevProjects/TriviaFlix/data/PreguntasYRespuestas.json'

In [3]:
class MiProcesadorNLP:
    def __init__(self, temperatura=0.5, max_tokens=100, top_p=1.0, frequency_penalty=0.0, presence_penalty=0.0):
        self.template = """{question}"""
        self.prompt = PromptTemplate(input_variables=["question"], template=self.template)
        self.llm = OpenAI(temperature=temperatura, max_tokens=max_tokens, top_p=top_p, frequency_penalty=frequency_penalty, presence_penalty=presence_penalty)
        self.chain = LLMChain(llm=self.llm, prompt=self.prompt)
        
    @tiempo_de_ejecucion
    def ejecutar(self, question):
        return self.chain.run(question=question)

In [4]:
procesador = MiProcesadorNLP()
resultado = procesador.ejecutar(question="¿Cuál es la capital de Francia?")
logging.info(f"Resultado: {resultado}")

In [5]:
import json

# Configuración del logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class ProcesadorJson:
    def __init__(self, ruta_archivo):
        self.ruta_archivo = ruta_archivo

    def cargar_json(self):
        """
        Carga un archivo JSON y devuelve su contenido.
        """
        try:
            with open(self.ruta_archivo, "r") as f:
                data = json.load(f)
            logging.info(f"Archivo JSON cargado con éxito: '{self.ruta_archivo}'.")
            return data
        except Exception as e:
            logging.error(f"Error al cargar el archivo JSON: {e}")
            return None

    def preparar_texto(self):
        """
        Extrae títulos y opciones de las preguntas del JSON, concatenándolos en una única cadena de texto.
        """
        data = self.cargar_json()
        if data:
            texto_completo = ' '.join([
                f"{item['titulo']} Opción A: {item['opcionA']} Opción B: {item['opcionB']} " +
                f"Opción C: {item['opcionC']} Opción D: {item['opcionD']}" 
                for item in data
            ])
            logging.info("Texto preparado con éxito a partir de JSON.")
            return texto_completo
        else:
            logging.error("No se pudo cargar datos para preparar texto.")
            return ""

In [6]:
# Uso de ProcesadorJson para cargar un archivo JSON

procesador_json = ProcesadorJson(ruta_archivo=json_path)
texto_completo = procesador_json.preparar_texto()
print(texto_completo)

In [7]:
from langchain.text_splitter import CharacterTextSplitter
import logging

class SeparadorTexto:
    def __init__(self, separator=".", chunk_size=1000, chunk_overlap=200):
        self.text_splitter = CharacterTextSplitter(
            separator=separator,
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
        )
    
    def dividir_texto(self, texto):
        """
        Divide el texto en segmentos más pequeños basados en la configuración inicial.
        """
        try:
            texts = self.text_splitter.split_text(texto)
            logging.info(f"Texto dividido en {len(texts)} segmentos.")
            return texts
        except Exception as e:
            logging.error(f"Error al dividir texto: {e}")
            return []

In [8]:
# Integración con el flujo de trabajo

if texto_completo:
    separador = SeparadorTexto()
    segmentos_texto = separador.dividir_texto(texto_completo)
    
    if segmentos_texto:
        # Procesar cada segmento con `MiProcesadorNLP` u otra lógica específica
        # Por ejemplo, imprimir el primer segmento
        print(segmentos_texto[0])
    else:
        print("No se pudo dividir el texto.")
else:
    print("No se pudo extraer texto del archivo JSON.")

In [9]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import logging

class BuscadorDocumentos:
    def __init__(self):
        self.embeddings = OpenAIEmbeddings()
        self.docsearch = None  # Se inicializará después con los textos
    
    def inicializar_busqueda(self, texts):
        """
        Inicializa el motor de búsqueda FAISS con los textos proporcionados.
        """
        try:
            self.docsearch = FAISS.from_texts(texts, self.embeddings)
            logging.info("Motor de búsqueda FAISS inicializado con éxito.")
        except Exception as e:
            logging.error(f"Error al inicializar FAISS: {e}")
            self.docsearch = None
    
    def buscar(self, query, top_n=5, search_type='similarity'):
        """
        Realiza una búsqueda en el motor FAISS y devuelve los top_n resultados.
        Se espera que search_type sea 'similarity' o 'mmr'.
        """
        if not self.docsearch:
            logging.error("Motor de búsqueda FAISS no inicializado.")
            return []
        try:
            resultados = self.docsearch.search(query, top_n=top_n, search_type=search_type)
            logging.info(f"Búsqueda realizada con éxito. {len(resultados)} resultados encontrados.")
            return resultados
        except Exception as e:
            logging.error(f"Error durante la búsqueda: {e}")
            return []

In [10]:
buscador = BuscadorDocumentos()
buscador.inicializar_busqueda(segmentos_texto)

# Realizar una búsqueda de ejemplo
resultados_busqueda = buscador.buscar("Consulta de ejemplo", top_n=3)
if resultados_busqueda:
    print("Resultados de la búsqueda:", resultados_busqueda)
else:
    print("No se encontraron resultados.")

In [11]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI


class ProcesadorQA:
    def __init__(self, buscador_documentos):
        self.buscador_documentos = buscador_documentos
        self.chain = load_qa_chain(OpenAI(), chain_type="stuff")
    
    def ejecutar_qa(self, query):
        """
        Ejecuta una consulta de preguntas y respuestas utilizando documentos
        relevantes obtenidos a través de una búsqueda de similitud.
        """
        try:
            # Realiza una búsqueda de similitud para encontrar documentos relevantes
            docs = self.buscador_documentos.docsearch.similarity_search(query)
            # Ejecuta la cadena QA con la pregunta y los documentos encontrados
            resultado = self.chain.run(question=query, input_documents=docs)
            logging.info("Consulta de QA ejecutada con éxito.")
            return resultado
        except Exception as e:
            logging.error(f"Error al ejecutar la consulta de QA: {e}")
            return None

In [12]:
# Creación y uso de ProcesadorQA
procesador_qa = ProcesadorQA(buscador)
resultado_qa = procesador_qa.ejecutar_qa(query="Haz un resumen de las preguntas y respuestas.")

if resultado_qa:
    print("Resultado de la consulta QA:", resultado_qa)
else:
    print("No se pudo ejecutar la consulta de QA.")

In [13]:
# Otra consulta de ejemplo

resultado_qa_2 = procesador_qa.ejecutar_qa(query="Nombra algunas de las series de netflix que se mencionan en el json.")
if resultado_qa_2:
    print("Resultado de la consulta QA:", resultado_qa_2)
else:
    print("No se pudo ejecutar la consulta de QA.")

In [14]:
import json

class ProcesadorTrivia:
    def __init__(self, datos_trivia, procesador_qa):
        self.trivia = datos_trivia
        self.procesador_qa = procesador_qa

    def ejecutar_trivia(self):
        """Ejecuta las preguntas de trivia y almacena las respuestas utilizando un Procesador QA."""
        respuestas = {}
        for pregunta in self.trivia:
            pregunta_texto = f"{pregunta['titulo']} Opción A: {pregunta['opcionA']} Opción B: {pregunta['opcionB']} " \
                             f"Opción C: {pregunta['opcionC']} Opción D: {pregunta['opcionD']}"
            # Simular la recuperación de respuesta utilizando ProcesadorQA
            respuesta = self.procesador_qa.ejecutar_qa(query=pregunta_texto)
            respuesta_correcta = pregunta[f"opcion{pregunta['correcta'].upper()}"]
            respuestas[pregunta["id"]] = {
                "pregunta": pregunta_texto,
                "respuesta_obtenida": respuesta,
                "respuesta_correcta": respuesta_correcta
            }
        return respuestas
    
    def guardar_respuestas(self, respuestas, ruta_archivo):
        """Guarda las respuestas de la trivia en un archivo JSON."""
        try:
            with open(ruta_archivo, "w") as f:
                json.dump(respuestas, f, indent=4)
            logging.info(f"Respuestas de la trivia guardadas en '{ruta_archivo}'.")
        except Exception as e:
            logging.error(f"Error al guardar respuestas de la trivia: {e}")

In [ ]:
datos_trivia = procesador_json.cargar_json()
procesador_trivia = ProcesadorTrivia(datos_trivia, procesador_qa)
respuestas_trivia = procesador_trivia.ejecutar_trivia()
procesador_trivia.guardar_respuestas(respuestas_trivia, "respuestas_trivia.json")